In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

D:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import json
with open('content.json') as content:
    data1=json.load(content)

In [4]:
tag=[]
inputs=[]
responses={}

for intent in data1['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['patterns']:
        inputs.append(lines)
        tag.append(intent['tag'])

In [5]:
data=pd.DataFrame({"inputs":inputs,"tags":tag})
data

,inputs,tags
0,Hi,greeting
1,How are you?,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Good day,greeting
...,...,...
363,ragging against,ragging
364,antiragging facility,ragging
365,ragging juniors,ragging
366,ragging history,ragging


In [6]:
data=data.sample(frac=1)

In [7]:
import string

data['inputs']=data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs']=data['inputs'].apply(lambda wrd: ''.join(wrd))
data

,inputs,tags
208,canteen facility,canteen
168,events organised,event
28,whats your name,name
161,cse lab,LabDetails
282,facilities,facilities
...,...,...
348,good work,salutaion
240,sem,sem
335,sports facilities,sports
80,what is name of aamec founder,Founder


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=2000)
t=tokenizer.fit_on_texts(data['inputs'])
train=tokenizer.texts_to_sequences(data['inputs'])


from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train=pad_sequences(train)


from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

y_train=le.fit_transform(data['tags'])
print(data['tags'])
print(y_train)


208       canteen
168         event
28           name
161    LabDetails
282    facilities
          ...    
348     salutaion
240           sem
335        sports
80        Founder
32           name
Name: tags, Length: 368, dtype: object
[20 23 33  7 24 19 42 28 28 35 34 34 26 45  0  3 45 32 41 19 20 35 25 21
 38 27 35 40 40 45 34 30 31  8 40 30 20 36 26 11 24 14 33 30 39 37 39 30
  5  8 41 40  0 25 18 30 16 43 40 26 41 28 37 13 33 22 28 23 26 20 46 40
 30 44 20 35 20 25 40 40 40 40 17 20 46 34 22 43 38 12 25 37 19 28 44 20
  2 19 15 30 38 43 33 37  6 28 33 34 15 45 46 46 44 23 46  2 34 43 10  7
 35 33 27 41 34 39  0 25 34 14 30 31 44 16 30 31 27  0 25 13 30 40 11  1
 37 46 32  3 12 16 13 22 37 35 39 11 39 32 23 16 13 18 17 33 39 23  8 40
  2  6 39 22 23 37 26 12 21 10 40 40 34  7 36 35 23 12 34 26 30  5 34 27
 28  2  1 19 35 28  5 15 23  5 31 18 40 34 37 20 16 34 46 34 26 34 24 40
 18 40 35 40  4 21 45 46 27 13 28 12 17 33 44 19 39 16 28 26 23 46 10 41
 33 26 41 37  4 18 14 32 21 41 42

In [9]:
input_shape=x_train.shape[1]
print(input_shape)

14


In [10]:
vocabulary=len(tokenizer.word_index)
print(tokenizer.word_index)
print('number of unquie words',vocabulary)

output_length=le.classes_.shape[0]
print('Output length ',output_length)

{'college': 1, 'is': 2, 'what': 3, 'of': 4, 'cse': 5, 'the': 6, 'aamec': 7, 'and': 8, 'are': 9, 'scholarship': 10, 'you': 11, 'how': 12, 'fees': 13, 'in': 14, 'library': 15, 'quota': 16, 'about': 17, 'i': 18, '2023': 19, 'when': 20, 'courses': 21, 'can': 22, 'there': 23, 'facilities': 24, 'to': 25, 'number': 26, 'for': 27, 'government': 28, 'canteen': 29, 'events': 30, 'name': 31, '2024': 32, 'have': 33, 'exam': 34, 'engineering': 35, 'departments': 36, 'graduate': 37, 'ragging': 38, 'do': 39, 'many': 40, 'facility': 41, 'your': 42, 'sports': 43, 'timing': 44, 'any': 45, 'me': 46, 'management': 47, 'pg': 48, 'first': 49, 'ug': 50, 'admission': 51, 'who': 52, 'uniform': 53, 'placement': 54, 'semester': 55, 'list': 56, 'programme': 57, 'details': 58, 'companies': 59, 'contact': 60, 'bus': 61, 'food': 62, 'available': 63, 'information': 64, '2022': 65, 'hod': 66, 'year': 67, 'where': 68, 'does': 69, 'committee': 70, 'holidays': 71, 'process': 72, 'infrastructure': 73, 'founder': 74, 'will

In [11]:
#creating model


i=Input(shape=(input_shape,))
x=Embedding(vocabulary+1,10)(i)
x=LSTM(10,return_sequences=True)(x)
x=Flatten()(x)
x=Dense(output_length,activation='softmax')(x)
model=Model(i,x)



In [12]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model.fit(x_train,y_train,epochs=200)

Epoch 1/200
12/12 [==============================] - 3s 9ms/step - loss: 3.8458 - accuracy: 0.0272
Epoch 2/200
12/12 [==============================] - 0s 8ms/step - loss: 3.8296 - accuracy: 0.0543
Epoch 3/200
12/12 [==============================] - 0s 8ms/step - loss: 3.8014 - accuracy: 0.0543
Epoch 4/200
12/12 [==============================] - 0s 8ms/step - loss: 3.7491 - accuracy: 0.0598
Epoch 5/200
12/12 [==============================] - 0s 9ms/step - loss: 3.6859 - accuracy: 0.0842
Epoch 6/200
12/12 [==============================] - 0s 8ms/step - loss: 3.6573 - accuracy: 0.0707
Epoch 7/200
12/12 [==============================] - 0s 10ms/step - loss: 3.6396 - accuracy: 0.0707
Epoch 8/200
12/12 [==============================] - 0s 8ms/step - loss: 3.6236 - accuracy: 0.0761
Epoch 9/200
12/12 [==============================] - 0s 8ms/step - loss: 3.6089 - accuracy: 0.0761
Epoch 10/200
12/12 [==============================] - 0s 8ms/step - loss: 3.5901 - accuracy: 0.0788
Epoch 11

KeyboardInterrupt: 

In [14]:
from flask import Flask,render_template,redirect,request,jsonify
from flask_ngrok import run_with_ngrok
import random

print(model)

def generate_res(prediction_input):
    
    texts_p=[]
  
    
    prediction_input=[letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input=''.join(prediction_input)
    texts_p.append(prediction_input)
    
    print(prediction_input)
    
    
    prediction_input=tokenizer.texts_to_sequences(texts_p)
    prediction_input=np.array( prediction_input).reshape(-1)
    prediction_input=pad_sequences([prediction_input],input_shape)
    
    output=model.predict(prediction_input)
    output=output.argmax()
  
    
    response_tag=le.inverse_transform([output])[0]
    response_tag=le.inverse_transform([output])[0]
    res=random.choice(responses[response_tag])
    print(f"HackAI ({response_tag}) :",res)
    return res


In [ ]:
app=Flask(__name__)
app.secret_key="12345"

run_with_ngrok(app)

@app.route('/')
def Home():
    return render_template('Home.html',titleName='HackAI Chatbot | Main Page')

@app.route('/chat',methods=['GET','POST'])
def ChatBot():
    res=0    
    if request.method == 'POST':
        message=str(request.form['userReply'])
        bot_response=generate_res(message)
        print('Message  -->',message)
    
        while True:

           if message!='':
               bot_response = str(bot_response)      
               print(bot_response)
               return jsonify({'status':'OK','answer':bot_response})
           elif message == "bye":
               bot_response='Hope to see you soon'
               print(bot_response)
               return jsonify({'status':'OK','answer':bot_response})
               break

    return render_template('chat.html',titleName='HackAI Chatbot | Chat')

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4937-2401-4900-4844-d585-c66-dfff-ff2a-c761.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Sep/2023 10:49:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2023 10:49:19] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Sep/2023 10:49:21] "GET /chat HTTP/1.1" 200 -


principal
1/1 [==============================] - 2s 2s/step


...

127.0.0.1 - - [15/Sep/2023 10:55:11] "POST /chat HTTP/1.1" 200 -


HackAI (greeting) : Hello!
Message  --> how are you
Hello!
what is your name
1/1 [==============================] - 0s 68ms/step


127.0.0.1 - - [15/Sep/2023 10:55:40] "POST /chat HTTP/1.1" 200 -


HackAI (name) : I am your helper
Message  --> what is your name?
I am your helper


127.0.0.1 - - [15/Sep/2023 10:55:50] "GET /chat HTTP/1.1" 200 -


In [ ]:
import random

while True:
    
    texts_p=[]
    prediction_input=input('You :')
    
    prediction_input=[letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input=''.join(prediction_input)
    texts_p.append(prediction_input)
    
    print(prediction_input)
    
    
    prediction_input=tokenizer.texts_to_sequences(texts_p)
    prediction_input=np.array( prediction_input).reshape(-1)
    prediction_input=pad_sequences([prediction_input],input_shape)
    
    output=model.predict(prediction_input)
    output=output.argmax()
  
    
    response_tag=le.inverse_transform([output])[0]
    
#     print('Response Tag -->',response_tag)
    print(f"HackAI ({response_tag}) :",random.choice(responses[response_tag]))
    
    if response_tag == 'goodbye':
        break
        